<a href="https://colab.research.google.com/github/BibhuDas123/Data-Science-Projects/blob/master/RAPIDS_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#we are going to run on gpu basis for xgboost.As xgboost is not natively supported,so we will use java binding and integrate with spark.GPU is used for heavy computations and spark itself is disturbed system 
# and does operationd in row by row basis and incase you want to accumalte rows  into memeory and do vectorisation we can use spark and GPU vectorisation for first processing and better perofromance.
!nvidia-smi

Wed Jan  6 11:24:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)
if (device_name != b'Tesla T4') and (device_name != b'Tesla P4') and (device_name != b'Tesla P100-PCIE-16GB'):
  raise Exception("""
    Unfortunately this instance does not have a T4, P4 or P100 GPU.
    
    Please make sure you've configured Colab to request a GPU instance type.
    
    Sometimes Colab allocates a Tesla K80 instead of a T4, P4 or P100. Resetting the instance.
If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!')

Woo! You got the right kind of GPU!


In [ ]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!bash rapidsai-csp-utils/colab/rapids-colab.sh stable







fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
PLEASE READ
********************************************************************************************************
Changes:
1. IMPORTANT CHANGES: RAPIDS on Colab will be pegged to 0.14 Stable until further notice.
2. Default stable version is now 0.14.  Nightly will redirect to 0.14.
3. You can now declare your RAPIDSAI version as a CLI option and skip the user prompts (ex: '0.14' or '0.15', between 0.13 to 0.14, without the quotes): 
        "!bash rapidsai-csp-utils/colab/rapids-colab.sh <version/label>"
        Examples: '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.14', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh stable', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh s'
                  '!bash rapidsai-csp-utils/colab/rapids-colab.sh 0.15, or '!bash rapidsai-csp-utils/colab/rapids-colab.sh nightly', or '!bash rapidsai-csp-utils/colab/rapids-colab.sh n'
Enjoy using RAPIDS!  

In [4]:
import sys, os

dist_package_index = sys.path.index('/usr/local/lib/python3.6/dist-packages')
sys.path = sys.path[:dist_package_index] + ['/usr/local/lib/python3.6/site-packages'] + sys.path[dist_package_index:]
sys.path
exec(open('rapidsai-csp-utils/colab/update_modules.py').read(), globals())

***********************************************************************
Let us check on those pyarrow and cffi versions...
***********************************************************************

You're don't have pyarrow.
unloaded cffi 1.14.4
loaded cffi 1.14.4


In [ ]:

#only for 0.10
!conda install -y --prefix /usr/local -c rapidsai/label/xgboost -c rapidsai -c nvidia -c conda-forge dask-cudf xgboost

Solving environment: - \ | / - 

In [ ]:
!ls /usr/lib/libxgboost.so

In [ ]:

!cp /usr/local/lib/libxgboost.so /usr/lib

In [ ]:
 !pip install  llvmlite
 !pip3 install numba==0.43.1

 

In [ ]:
import llvmlite
import numba

In [ ]:
import cudf

In [ ]:
from __future__ import print_function
import sys,tempfile, urllib, os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.datasets import fetch_openml
covtyp = fetch_openml(name='covertype', version=4)

In [ ]:
covtyp.data.shape
np.unique(covtyp.target)

In [ ]:
!nvidia-smi

In [ ]:
cov_df = pd.DataFrame(data= np.c_[covtyp['data'], covtyp['target']],
                     columns= covtyp['feature_names'] + ['target'])

In [ ]:
for cols in cov_df.columns:
  cov_df[cols] = pd.to_numeric(cov_df[cols], downcast='float')

In [ ]:

cov_df['target'] = cov_df['target'].astype(np.int32)

cov_df['target'] = cov_df['target']-1


cov_df_X = cov_df.copy()
cov_df_y =  cov_df_X.pop('target')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(cov_df_X, cov_df_y, train_size=0.95, test_size=0.05)

In [ ]:
X_train_gdf = cudf.DataFrame.from_pandas(X_train)
X_test_gdf = cudf.DataFrame.from_pandas(X_test)
y_train_gdf = cudf.DataFrame.from_pandas(pd.DataFrame(y_train))
y_test_gdf = cudf.DataFrame.from_pandas(pd.DataFrame(y_test))

In [ ]:
!nvidia-smi

In [ ]:

from cuml import RandomForestClassifier as cuRF
import time

In [ ]:

cu_rf_params = {
    'n_estimators': 250,
    'max_depth': 8,
    'n_streams': 1,
    'split_algo': 0, 
    'seed': 1010
}
cu_rf = cuRF(**cu_rf_params)

In [ ]:
start_time = time.time()
cu_rf.fit(X_train_gdf, y_train_gdf)
print("GPU Training Time with GPU dataframe: %s seconds" % (str(time.time() - start_time)))

In [ ]:
predvalue = cu_rf.predict(X_test_gdf)

In [ ]:
:
#cu_rf.score(X_test_gdf, y_test_gdf) --> use GPU frame if using 0.11 version of rapids
cu_rf.score(X_test_gdf, y_test.to_numpy())

In [ ]:
pred_out=predvalue.copy_to_host()

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred_out)
